# Lab.5: Lexical semantics
## Introduction to Human Language Technologies
### Victor Badenas Crespo

***

### Statement

- Read all pairs of sentences of the trial set within the evaluation framework of the project.
- Apply Lesk’s algorithm to the words in the sentences.
- Compute their similarities by considering senses and Jaccard coefficient.
- Compare the results with those in session 2 (document) and 3 (morphology) in which words and lemmas were considered.
- Compare the results with gold standard by giving the pearson correlation between them.

***

## Solution

In [1]:
# core imports
from pprint import pprint
from pathlib import Path
from collections import Counter

# scipy imports
from scipy.stats import pearsonr

# nltk imports
import nltk
from nltk.metrics import jaccard_distance
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk

# nltk downloads
nltk.download('punkt')
nltk.download('wordnet')

# constants definition
DATA_FOLDER = Path('./trial')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\victo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\victo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


First functions for reading and structuring the data are declared, then the input data is read which has multiple lines containing \[id, sentence1, sentence2\]. The Gold standard info is also read. Then the inputText is formatted into a dict object with the following format for readability:

```json
{
    "id": <id_string>,
    "sent1": <sentence_string>,
    "sent2": <sentence_string>
}
```

In [2]:
def readFile(filePath):
    """
    reads and returns a list of lists containing the text split by line 
    jumps and by tab characters
    """
    with open(filePath, 'r') as fileHandler:
        data = fileHandler.readlines()
    
    # split every line by tabs
    data = list(map(lambda x: x.strip().split('\t'), data))
    return data

def toDict(line):
    """
    creates a dict with fields id sent1 sent2 from the values in line
    """
    keys = ("id", "sent1", "sent2")
    return dict(zip(keys, line))

# read file data
inputText = readFile(DATA_FOLDER / 'STS.input.txt')
gsText = readFile(DATA_FOLDER / 'STS.gs.txt')

# convert to previously defined dict structure
inputText = list(map(toDict, inputText))
pprint(inputText)

[{'id': 'id1',
  'sent1': 'The bird is bathing in the sink.',
  'sent2': 'Birdie is washing itself in the water basin.'},
 {'id': 'id2',
  'sent1': 'In May 2010, the troops attempted to invade Kabul.',
  'sent2': 'The US army invaded Kabul on May 7th last year, 2010.'},
 {'id': 'id3',
  'sent1': 'John said he is considered a witness but not a suspect.',
  'sent2': '"He is not a suspect anymore." John said.'},
 {'id': 'id4',
  'sent1': 'They flew out of the nest in groups.',
  'sent2': 'They flew into the nest together.'},
 {'id': 'id5',
  'sent1': 'The woman is playing the violin.',
  'sent2': 'The young lady enjoys listening to the guitar.'},
 {'id': 'id6',
  'sent1': 'John went horse back riding at dawn with a whole group of friends.',
  'sent2': 'Sunrise at dawn is a magnificent view to take in if you wake up '
           'early enough for it.'}]


In [5]:
for sentence in inputText:
    id, sent1, sent2 = sentence["id"], sentence["sent1"], sentence["sent2"]
    sent1 = nltk.word_tokenize(sent1)
    sent2 = nltk.word_tokenize(sent2)
    contextSentences1 = list(map(lambda word: lesk(sent1, word), sent1))
    contextSentences2 = list(map(lambda word: lesk(sent1, word), sent2))
    contextSentences1 = list(filter(lambda i: i is not None, contextSentences1))
    contextSentences2 = list(filter(lambda i: i is not None, contextSentences2))
    contextSentences1 = list(map(lambda i: i.definition(), contextSentences1))    
    contextSentences2 = list(map(lambda i: i.definition(), contextSentences2))

    sentence["contextSentences1"] = contextSentences1
    sentence["contextSentences2"] = contextSentences2

In [6]:
pprint(inputText)

[{'contextSentences1': ['the flesh of a bird or fowl (wild or domestic) used '
                        'as food',
                        'to remain unmolested, undisturbed, or uninterrupted '
                        '-- used only in infinitive form',
                        'immersing the body in water or sunshine',
                        'a rare soft silvery metallic element; occurs in small '
                        'quantities in sphalerite',
                        'a depression in the ground communicating with a '
                        'subterranean passage (especially in limestone) and '
                        'formed by solution or by collapse of a cavern roof'],
  'contextSentences2': ['shoot in one stroke under par',
                        'to remain unmolested, undisturbed, or uninterrupted '
                        '-- used only in infinitive form',
                        'remove by the application of water or other liquid '
                        'and soap or some o

In [18]:
def computeSimilarity(sentenceDict):
    context1 = ' '.join(sentenceDict["contextSentences1"])
    context2 = ' '.join(sentenceDict["contextSentences2"])
    context1 = set(nltk.word_tokenize(context1, language='english'))
    context2 = set(nltk.word_tokenize(context2, language='english'))
    return jaccard_distance(context1, context2)

In [19]:
testDistances = list(map(computeSimilarity, inputText))

In [21]:
refDistances = [float(value)/(len(gsText)-1) for _, value in gsText]

pcorr = pearsonr(refDistances, testDistances)[0]

# formatting for purely demonstrative purposes
print(f"pearsonr({list(map(lambda x:float('%.2f' % x), refDistances))}, {list(map(lambda x:float('%.2f' % x), testDistances))}) = {pcorr}")

pearsonr([0.0, 0.2, 0.4, 0.6, 0.8, 1.0], [0.67, 0.61, 0.16, 0.45, 0.84, 0.73]) = 0.28131541901748136


*** 

## Conlusion

**placeholder**

***

### End of P4